---
title: "exam: Variable Selection and Regularization "
author: "Hanna Wierszok"
format: html
jupiter:python3
---

In [1039]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from plotnine import ggplot, geom_point, aes, geom_smooth, geom_histogram, geom_line, labs, scale_size, geom_hline, ggtitle, scale_color_manual

import matplotlib.pyplot as plt
import math

In [1040]:
train_data = pd.read_csv("data/train_new.csv")
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2197 entries, 0 to 2196
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SalePrice      2197 non-null   int64  
 1   PID            2197 non-null   int64  
 2   Lot Frontage   1835 non-null   float64
 3   Lot Area       2197 non-null   int64  
 4   Street         2197 non-null   object 
 5   Neighborhood   2197 non-null   object 
 6   Bldg Type      2197 non-null   object 
 7   House Style    2197 non-null   object 
 8   Overall Qual   2197 non-null   int64  
 9   Overall Cond   2197 non-null   int64  
 10  Year Built     2197 non-null   int64  
 11  Roof Style     2197 non-null   object 
 12  Heating        2197 non-null   object 
 13  Central Air    2197 non-null   object 
 14  Electrical     2196 non-null   object 
 15  Full Bath      2197 non-null   int64  
 16  Half Bath      2197 non-null   int64  
 17  Bedroom AbvGr  2197 non-null   int64  
 18  TotRms A

In [1041]:
test_data = pd.read_csv("data/test_new.csv")
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 605 entries, 0 to 604
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PID            605 non-null    int64 
 1   Lot Frontage   605 non-null    int64 
 2   Lot Area       605 non-null    int64 
 3   Street         605 non-null    object
 4   Neighborhood   605 non-null    object
 5   Bldg Type      605 non-null    object
 6   House Style    605 non-null    object
 7   Overall Qual   605 non-null    int64 
 8   Overall Cond   605 non-null    int64 
 9   Year Built     605 non-null    int64 
 10  Roof Style     605 non-null    object
 11  Heating        605 non-null    object
 12  Central Air    605 non-null    object
 13  Electrical     605 non-null    object
 14  Full Bath      605 non-null    int64 
 15  Half Bath      605 non-null    int64 
 16  Bedroom AbvGr  605 non-null    int64 
 17  TotRms AbvGrd  605 non-null    int64 
 18  Gr Liv Area    605 non-null   

In [1042]:
train_data['SalePrice'].describe()

count      2197.000000
mean     182376.851161
std       81168.157405
min       13100.000000
25%      130000.000000
50%      163500.000000
75%      215000.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [1043]:
# features preprocessing
train_data_cl=train_data.drop(['PID','SalePrice'], axis=1)
test_data_cl=test_data.drop(['PID'], axis=1)

# Combine train and test features in order to apply the feature transformation pipeline to the entire dataset
ntrain = train_data_cl.shape[0]
ntest = test_data_cl.shape[0]
all_features = pd.concat([train_data_cl, test_data_cl]).reset_index(drop=True)
all_features.shape

(2802, 23)

In [1044]:
#Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
all_features['Lot Frontage'] = all_features.groupby('Neighborhood')['Lot Frontage'].transform(lambda x: x.fillna(x.median()))
all_features['Lot Frontage'] = all_features['Lot Frontage'].transform(lambda x: x.fillna(x.median()))

In [1045]:
#It has one NA value. Since this feature has mostly 'SBrkr', we can set that for the missing value.
all_features['Electrical'] = all_features['Electrical'].fillna("SBrkr")
#all_features.info()

In [1046]:
#Year  sold are transformed into categorical features.
all_features['Yr Sold'] = all_features['Yr Sold'].astype(str)
#all_features.info()

In [1048]:
#add some new features
all_features['HasScreen Porch'] = all_features['Screen Porch'].apply(lambda x: 1 if x > 0 else 0)
all_features['Total_Home_Quality'] = all_features['Overall Qual'] + all_features['Overall Cond']
all_features['Total_Bathrooms'] = all_features['Full Bath'] + (0.5 * all_features['Half Bath'])
all_features['HasPool'] = all_features['Pool Area'].apply(lambda x: 1 if x > 0 else 0)
all_features.shape

(2802, 27)

In [1049]:
all_features = all_features.drop(['Heating','Functional','Sale Type','Electrical','Street','Neighborhood'], axis=1) 
all_features.shape

(2802, 21)

In [1050]:
# Identify columns for preprocessing

categorical_cols = all_features.select_dtypes(include=['object']).columns
categorical_cols

Index(['Bldg Type', 'House Style', 'Roof Style', 'Central Air', 'Yr Sold'], dtype='object')

In [1051]:
# Identify columns for preprocessing
feature_skew = all_features.select_dtypes(include=[np.number]).skew()
#feature_skew

In [1052]:
feature_skew=feature_skew.drop(labels=['HasScreen Porch', 'HasPool'])

In [1053]:
log_colums = feature_skew[abs(feature_skew) > 0.5].index
log_colums

Index(['Lot Frontage', 'Lot Area', 'Overall Cond', 'Year Built', 'Half Bath',
       'TotRms AbvGrd', 'Gr Liv Area', 'Screen Porch', 'Pool Area',
       'Total_Home_Quality'],
      dtype='object')

In [1054]:
scale_colums = [name for name in feature_skew.index if name not in log_colums]
scale_colums

['Overall Qual', 'Full Bath', 'Bedroom AbvGr', 'Total_Bathrooms']

In [1055]:
#add log1 of Gr Liv Area
all_features['Gr Liv Area_log1'] = np.log1p(all_features['Gr Liv Area'])
all_features['Year Built_log1'] = np.log1p(all_features['Year Built'])
all_features['Overall Qual_log1'] = np.log1p(all_features['Overall Qual'])
r2_colums=['Gr Liv Area_log1','Year Built_log1','Overal Qual_log1']
all_features.shape

(2802, 24)

In [1056]:
folders=5
#split 
train_data_cl = all_features[:ntrain]
test_data_cl = all_features[ntrain:]
train_data_cl

,Lot Frontage,Lot Area,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Roof Style,Central Air,Full Bath,...,Screen Porch,Pool Area,Yr Sold,HasScreen Porch,Total_Home_Quality,Total_Bathrooms,HasPool,Gr Liv Area_log1,Year Built_log1,Overall Qual_log1
0,80.0,9605,1Fam,1Story,7,6,2007,Gable,Y,1,...,0,0,2009,0,13,1.5,0,7.105786,7.604894,2.079442
1,90.0,14684,1Fam,1Story,7,7,1990,Hip,Y,2,...,0,0,2009,0,14,2.0,0,7.694848,7.596392,2.079442
2,82.0,14375,1Fam,SLvl,6,6,1958,Gable,Y,1,...,233,0,2009,1,12,1.0,0,7.204149,7.580189,1.945910
3,48.0,6472,TwnhsE,1Story,9,5,2008,Hip,Y,2,...,0,0,2009,0,14,2.0,0,7.284135,7.605392,2.302585
4,61.0,9734,1Fam,SLvl,7,5,2004,Gable,Y,2,...,0,0,2009,0,12,2.5,0,7.226209,7.603399,2.079442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2192,80.0,10041,1Fam,2Story,8,5,1992,Gable,Y,2,...,0,0,2006,0,13,2.5,0,7.557995,7.597396,2.197225
2193,70.0,6300,1Fam,1.5Fin,5,4,1938,Gable,Y,1,...,0,0,2009,0,9,1.5,0,7.145984,7.569928,1.791759
2194,41.0,12460,1Fam,2Story,7,5,1999,Gable,Y,2,...,0,0,2008,0,12,2.5,0,7.750615,7.600902,2.079442
2195,85.0,10625,1Fam,1Story,5,5,1920,Gable,N,1,...,0,0,2010,0,10,1.0,0,6.728629,7.560601,1.791759


In [1057]:
test_data_cl

,Lot Frontage,Lot Area,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Roof Style,Central Air,Full Bath,...,Screen Porch,Pool Area,Yr Sold,HasScreen Porch,Total_Home_Quality,Total_Bathrooms,HasPool,Gr Liv Area_log1,Year Built_log1,Overall Qual_log1
2197,60.0,8070,1Fam,1Story,4,5,1994,Gable,Y,1,...,0,0,2007,0,9,1.0,0,6.898715,7.598399,1.609438
2198,40.0,6792,TwnhsE,1Story,7,5,2005,Gable,Y,2,...,0,0,2006,0,12,2.0,0,7.221836,7.603898,2.079442
2199,44.0,6371,TwnhsE,1Story,7,5,2009,Gable,Y,2,...,0,0,2010,0,12,2.0,0,7.214504,7.605890,2.079442
2200,70.0,8304,1Fam,2Story,6,5,1997,Gable,Y,2,...,0,0,2006,0,11,2.5,0,7.516433,7.599902,1.945910
2201,37.0,6951,1Fam,1Story,5,5,1984,Gable,Y,1,...,0,0,2008,0,10,1.0,0,6.828712,7.593374,1.791759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2797,34.0,5381,Twnhs,1Story,6,5,2005,Gable,Y,2,...,0,0,2009,0,11,2.0,0,7.175490,7.603898,1.945910
2798,64.0,13053,1Fam,1.5Fin,6,7,1923,Gambrel,Y,1,...,220,0,2008,1,13,1.5,0,7.522400,7.562162,1.945910
2799,53.0,6360,1Fam,1.5Fin,5,6,1942,Gable,Y,1,...,148,0,2010,1,11,1.5,0,7.282074,7.571988,1.791759
2800,43.0,7000,1Fam,2Story,7,8,1926,Gable,Y,1,...,0,0,2006,0,15,1.0,0,7.299797,7.563720,2.079442


In [1058]:
# Define predictors and target
X = train_data_cl#.drop(['Heating','Functional','Sale Type','Electrical','Street','Neighborhood'], axis=1) 
#y = train_data_cl['SalePrice']
y = np.log(train_data['SalePrice'])
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2197 entries, 0 to 2196
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Lot Frontage        2197 non-null   float64
 1   Lot Area            2197 non-null   int64  
 2   Bldg Type           2197 non-null   object 
 3   House Style         2197 non-null   object 
 4   Overall Qual        2197 non-null   int64  
 5   Overall Cond        2197 non-null   int64  
 6   Year Built          2197 non-null   int64  
 7   Roof Style          2197 non-null   object 
 8   Central Air         2197 non-null   object 
 9   Full Bath           2197 non-null   int64  
 10  Half Bath           2197 non-null   int64  
 11  Bedroom AbvGr       2197 non-null   int64  
 12  TotRms AbvGrd       2197 non-null   int64  
 13  Gr Liv Area         2197 non-null   int64  
 14  Screen Porch        2197 non-null   int64  
 15  Pool Area           2197 non-null   int64  
 16  Yr Sol

### preprocessing

In [1059]:
# Create a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('polynomial1',PolynomialFeatures(), ['Gr Liv Area_log1']),
        ('polynomial2',PolynomialFeatures(), ['Year Built_log1']),
         ('polynomial3',PolynomialFeatures(), ['Overall Qual_log1']),
        ('scale', StandardScaler(), scale_colums),
        ('log', FunctionTransformer(np.log1p, validate=False), log_colums),
        ('cat', OneHotEncoder(), categorical_cols)
    ],
     #remainder = "drop"
     remainder='passthrough'
)

### Linear regression

In [1060]:
# Create a pipeline with preprocessing and linear regression model
lr_pipeline = Pipeline([
     ('preprocessor', preprocessor),
      ('regressor', LinearRegression())
])
lr_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('polynomial1',
                                                  PolynomialFeatures(),
                                                  ['Gr Liv Area_log1']),
                                                 ('polynomial2',
                                                  PolynomialFeatures(),
                                                  ['Year Built_log1']),
                                                 ('polynomial3',
                                                  PolynomialFeatures(),
                                                  ['Overall Qual_log1']),
                                                 ('scale', StandardScaler(),
                                                  ['Overall Qual', 'Full Bath',
                                                   'Bedroom AbvGr',
                                                   'Total_Bathrooms']),
                                                 ('log',
                                                  FunctionTransformer(func=<ufunc 'log1p'>),
                                                  Index(['Lot Frontage', 'Lot Area', 'Overall Cond', 'Year Built', 'Half Bath',
       'TotRms AbvGrd', 'Gr Liv Area', 'Screen Porch', 'Pool Area',
       'Total_Home_Quality'],
      dtype='object')),
                                                 ('cat', OneHotEncoder(),
                                                  Index(['Bldg Type', 'House Style', 'Roof Style', 'Central Air', 'Yr Sold'], dtype='object'))])),
                ('regressor', LinearRegression())])

In [1061]:
degrees = {'preprocessor__polynomial1__degree': np.arange(1, 6),
           'preprocessor__polynomial2__degree': np.arange(1, 4),
           'preprocessor__polynomial3__degree': np.arange(1, 7)
          }

gscv = GridSearchCV(lr_pipeline, degrees, cv = folders, scoring='neg_mean_squared_error')
gscv_fitted = gscv.fit(X, y)
#tmp=pd.DataFrame(data = {'degree': degrees['degree'], "mse_scores": gscv_fitted.cv_results_['mean_test_score']*-1})
#tmp

In [1062]:
best_params = gscv_fitted.best_params_
best_params

{'preprocessor__polynomial1__degree': 5,
 'preprocessor__polynomial2__degree': 3,
 'preprocessor__polynomial3__degree': 6}

In [1063]:
print ('The MSE for Linear Regression: ', -gscv_fitted.best_score_)

The MSE for Linear Regression:  0.019196733549078356


In [1064]:
preprocessor = ColumnTransformer(
    transformers=[
        ('polynomial1',PolynomialFeatures(degree= best_params['preprocessor__polynomial1__degree']), ['Gr Liv Area_log1']),
        ('polynomial2',PolynomialFeatures(degree= best_params['preprocessor__polynomial2__degree'] ), ['Year Built_log1']),
         ('polynomial3',PolynomialFeatures( degree= best_params['preprocessor__polynomial3__degree']), ['Overall Qual_log1']),
        ('scale', StandardScaler(), scale_colums),
        ('log', FunctionTransformer(np.log1p, validate=False), log_colums),
        ('cat', OneHotEncoder(), categorical_cols)
    ],
     #remainder = "drop"
     remainder='passthrough'
)
# Create a pipeline with preprocessing and linear regression model
lr_pipeline = Pipeline([
     ('preprocessor', preprocessor),
      ('regressor', LinearRegression())
])

In [1065]:
# Fit the pipeline to the full dataset
lr_pipeline.fit(X, y)

# Extract coefficients from the linear regression model
model_coefficients = lr_pipeline.named_steps['regressor'].coef_

# Map coefficients to their corresponding feature names
feature_names = np.concatenate( [r2_colums,
                                scale_colums,
                                log_colums,
                                preprocessor.named_transformers_['cat'].get_feature_names_out()])
#feature_names = np.concatenate([numerical_cols, preprocessor.named_transformers_['cat'].get_feature_names_out()])
coefficients = dict(zip(feature_names, model_coefficients))

# Sort coefficients by absolute value
sorted_coefficients = dict(sorted(coefficients.items(), key=lambda item: abs(item[1]), reverse=True))
sorted_coefficients

{'Lot Frontage': 183244.5586749252,
 'House Style_1Story': 183244.55272329284,
 'Lot Area': -48437.61137814678,
 'Bldg Type_1Fam': -16734.478306488265,
 'Half Bath': 10885.226449952814,
 'TotRms AbvGrd': 7553.862308307946,
 'House Style_2Story': -2973.423567697042,
 'Year Built_log1': -2968.0087024037157,
 'Overall Cond': 2133.973751867663,
 'Overal Qual_log1': 1678.2917535848812,
 'Screen Porch': 1642.3679772496253,
 'Pool Area': -303.52654626256844,
 'Overall Qual': -236.46020052743881,
 'Gr Liv Area': 95.07050692152552,
 'Total_Home_Quality': 83.32993957038707,
 'Full Bath': 16.618894285572914,
 'Roof Style_Flat': -1.9272475639318145,
 'House Style_1.5Unf': 1.3918443478178233,
 'Bedroom AbvGr': -0.46610432907232313,
 'House Style_2.5Fin': 0.18315934793645283,
 'Bldg Type_Twnhs': -0.13795186586503405,
 'Bldg Type_2fmCon': 0.129320900014136,
 'Gr Liv Area_log1': 0.1197838967636891,
 'House Style_1.5Fin': 0.11703866578318411,
 'Yr Sold_2007': -0.1123777081653543,
 'Yr Sold_2010': 0.105

In [1066]:
# Perform cross-validation to estimate MSE
mse_scores = -cross_val_score(lr_pipeline, X, y, cv=folders, scoring='neg_mean_squared_error')
mse_scores

array([0.01915417, 0.01976671, 0.01832272, 0.02038641, 0.01835365])

In [1067]:
print ('The MSE for Linear Regression: ', mse_scores.mean())

The MSE for Linear Regression:  0.019196733549078356


### prediction

In [1069]:
test_data_cl
X_test=test_data_cl

In [1070]:
y_pred=lr_pipeline.predict(X_test)

In [1071]:
submission = pd.DataFrame()
submission['PID'] = test_data['PID']
submission['SalePrice']=np.exp(y_pred)
submission

,PID,SalePrice
0,907135180,123797.439694
1,528181040,198983.503831
2,528175010,203516.785617
3,531379030,194337.376541
4,923275090,120753.232568
...,...,...
600,528174060,180086.827251
601,903400180,189356.712426
602,903227150,135850.124632
603,909250070,152584.799580


In [1072]:
train_data['SalePrice'].describe()

count      2197.000000
mean     182376.851161
std       81168.157405
min       13100.000000
25%      130000.000000
50%      163500.000000
75%      215000.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [1073]:
submission['SalePrice'].describe()

count       605.000000
mean     172981.529432
std       72954.529398
min       52113.161784
25%      123667.114891
50%      150318.025043
75%      201160.021229
max      538689.575411
Name: SalePrice, dtype: float64

In [1074]:
submission.to_csv('submission2.csv', index=False)

In [1075]:
check_data = pd.read_csv("data/sample_submission_new.csv")
check_data

,PID,SalePrice
0,907135180,119752.701924
1,528181040,197375.060922
2,528175010,198338.379947
3,531379030,193227.786289
4,923275090,125498.537591
...,...,...
600,528174060,153642.542638
601,903400180,180701.850840
602,903227150,132755.981781
603,909250070,155980.687869


In [1080]:
check_data2 = pd.read_csv("submission2.csv")
check_data2

,PID,SalePrice
0,907135180,123797.439694
1,528181040,198983.503831
2,528175010,203516.785617
3,531379030,194337.376541
4,923275090,120753.232568
...,...,...
600,528174060,180086.827251
601,903400180,189356.712426
602,903227150,135850.124632
603,909250070,152584.799580


In [1082]:
check_data1 = pd.read_csv("submission1.csv")
check_data1

,PID,SalePrice
0,907135180,116856.857887
1,528181040,204955.116289
2,528175010,203570.955045
3,531379030,193396.832272
4,923275090,117870.760840
...,...,...
600,528174060,173931.728070
601,903400180,179164.118311
602,903227150,134019.143462
603,909250070,164017.532455


In [1079]:
MSE = mean_squared_error(np.log(check_data['SalePrice']), y_pred)
RMSE = math.sqrt(MSE)
print("Root Mean Square Error:\n")
print(RMSE)

Root Mean Square Error:

0.09109878375211111
